In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from wordpress_graph.paths import DATA_DIR, PROJECT_DIR
import feedparser
import numpy as np
from tqdm.autonotebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
#G = nx.read_gpickle((DATA_DIR/"network_path.gpickle").as_posix())
G = nx.karate_club_graph()

### Cluster communiities

In [4]:
import community
import seaborn as sns

In [5]:
partition = community.best_partition(G)
set(partition.values())

{0, 1, 2, 3}

### Create dictionary of graph

In [6]:
graph_data = { 'nodes': [], 'links': [] }
for nodei, node in enumerate(G.nodes()):
    graph_data['nodes'].append({
            "name": node,
            "group":  partition[node]

        })
node_num = {node['name']:nodei for nodei, node in enumerate(graph_data['nodes'])}
for edge, edge_val in G.edges().items():
    graph_data['links'].append({
        "source" : node_num[edge[0]],
        "target" : node_num[edge[1]],
        #"value" : edge_val['weight']
    })

In [10]:
graph_data

{'nodes': [{'name': 0, 'group': 0},
  {'name': 1, 'group': 0},
  {'name': 2, 'group': 0},
  {'name': 3, 'group': 0},
  {'name': 4, 'group': 1},
  {'name': 5, 'group': 1},
  {'name': 6, 'group': 1},
  {'name': 7, 'group': 0},
  {'name': 8, 'group': 2},
  {'name': 9, 'group': 2},
  {'name': 10, 'group': 1},
  {'name': 11, 'group': 0},
  {'name': 12, 'group': 0},
  {'name': 13, 'group': 0},
  {'name': 14, 'group': 2},
  {'name': 15, 'group': 2},
  {'name': 16, 'group': 1},
  {'name': 17, 'group': 0},
  {'name': 18, 'group': 2},
  {'name': 19, 'group': 0},
  {'name': 20, 'group': 2},
  {'name': 21, 'group': 0},
  {'name': 22, 'group': 2},
  {'name': 23, 'group': 3},
  {'name': 24, 'group': 3},
  {'name': 25, 'group': 3},
  {'name': 26, 'group': 2},
  {'name': 27, 'group': 3},
  {'name': 28, 'group': 3},
  {'name': 29, 'group': 2},
  {'name': 30, 'group': 2},
  {'name': 31, 'group': 3},
  {'name': 32, 'group': 2},
  {'name': 33, 'group': 2}],
 'links': [{'source': 0, 'target': 1},
  {'sourc

In [7]:
with open('graph.json', 'w') as f:
    json.dump(graph_data,
              f, indent=4,)

In [8]:
%%html
<div id="d3-example"></div>
<style>
.node {stroke: #fff; stroke-width: 1.5px;}
.link {stroke: #999; stroke-opacity: .6;}
</style>

In [9]:
%%javascript
// We load the d3.js library from the Web.
require.config({paths:
    {d3: "http://d3js.org/d3.v3.min"}});
require(["d3"], function(d3) {
  // The code in this block is executed when the
  // d3.js library has been loaded.

  // First, we specify the size of the canvas
  // containing the visualization (size of the
  // <div> element).
  var width = 300, height = 300;

  // We create a color scale.
  var color = d3.scale.category10();

  // We create a force-directed dynamic graph layout.
  var force = d3.layout.force()
    .charge(-120)
    .linkDistance(30)
    .size([width, height]);

  // In the <div> element, we create a <svg> graphic
  // that will contain our interactive visualization.
  var svg = d3.select("#d3-example").select("svg")
  if (svg.empty()) {
    svg = d3.select("#d3-example").append("svg")
          .attr("width", width)
          .attr("height", height);
  }

  // We load the JSON file.
  d3.json("graph.json", function(error, graph) {
    // In this block, the file has been loaded
    // and the 'graph' object contains our graph.

    // We load the nodes and links in the
    // force-directed graph.
    force.nodes(graph.nodes)
      .links(graph.links)
      .start();

    // We create a <line> SVG element for each link
    // in the graph.
    var link = svg.selectAll(".link")
      .data(graph.links)
      .enter().append("line")
      .attr("class", "link");

    // We create a <circle> SVG element for each node
    // in the graph, and we specify a few attributes.
    var node = svg.selectAll(".node")
      .data(graph.nodes)
      .enter().append("circle")
      .attr("class", "node")
      .attr("r", 5)  // radius
      .style("fill", function(d) {
         // The node color depends on the club.
         return color(d.group);
      })
      .call(force.drag);

    // The name of each node is the node number.
    node.append("title")
        .text(function(d) { return d.name; });

    // We bind the positions of the SVG elements
    // to the positions of the dynamic force-directed
    // graph, at each time step.
    force.on("tick", function() {
      link.attr("x1", function(d){return d.source.x})
          .attr("y1", function(d){return d.source.y})
          .attr("x2", function(d){return d.target.x})
          .attr("y2", function(d){return d.target.y});

      node.attr("cx", function(d){return d.x})
          .attr("cy", function(d){return d.y});
    });
  });
});

<IPython.core.display.Javascript object>